In [1]:
import os
import json
import pandas as pd

In [ ]:
# File paths (update as needed)
image_position_csv = r"D:\Downloads\client_187_Scooter.csv" 
position_parts_csv = "D:\major_parts_tw_scooter.csv"  
json_root_folder = r"D:\Downloads\TwoWheeler\data\NIC-2-wheeler-Batch-2-18\Download_data\testining_missing_code\nic_scooter"  

In [ ]:
# Load the CSV containing image names and positions
image_position_df =  pd.read_csv(image_position_csv)
image_position_df["image_name"] = image_position_df.url.str.split("/").str[-1]
image_position_df.rename(columns={'curation_result_y': 'position'}, inplace=True)

# Load the position-to-parts mapping
position_parts_df = pd.read_csv(position_parts_csv)

In [ ]:
position_part_mapping = {
    position.upper(): position_parts_df[position].dropna().unique().tolist()
    for position in position_parts_df.columns
}

def find_json_file(image_name):
    image_base = os.path.splitext(os.path.basename(image_name))[0]  
    for root, _, files in os.walk(json_root_folder):  
        for file in files:
            if file.endswith(".json") and file.startswith(image_base):
                return os.path.join(root, file)
    return None 


def check_missing_parts(image_name, position):
    json_path = find_json_file(image_name)  

    if json_path is None:
        return {"image_name": image_name, "position": position, "missing_parts": "JSON file not found"}

    with open(json_path, 'r') as f:
        data = json.load(f)

    required_parts = position_part_mapping.get(position.upper(), [])

    labeled_parts = {shape['label'] for shape in data.get('shapes', [])}

    missing_parts = list(set(required_parts) - labeled_parts)

    return {"image_name": image_name, "position": position, "missing_parts": missing_parts}

report = []

for _, row in image_position_df.iterrows():
    image_name = row['image_name']  
    position = row['position']

    if pd.isna(position):
        continue

    result = check_missing_parts(image_name, position)

    # Add to report if there are missing parts
    if result["missing_parts"]:
        report.append(result)

# Convert the report to a DataFrame and save it
report_df = pd.DataFrame(report)
report_csv = "missing_parts_report.csv"
report_df.to_csv(report_csv, index=False)

print(f"Missing parts report saved to {report_csv}")
